# Tese case: Sobl's G*-function

## Load modules

In [1]:
import numpy as np 
import numba as nb
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

## Define the Sobol's G*-function

In [2]:
def evaluate(values, delta, alpha, a):
    """Sobol G*-function.

    .. [1] Saltelli, A., Annoni, P., Azzini, I., Campolongo, F., Ratto, M., 
           Tarantola, S., 2010. Variance based sensitivity analysis of model 
           output. Design and estimator for the total sensitivity index. 
           Computer Physics Communications 181, 259–270. 
           https://doi.org/10.1016/j.cpc.2009.09.018

    Parameters
    ----------
    values : numpy.ndarray
        input variables
    delta : numpy.ndarray
        parameter values
    alpha : numpy.ndarray
        parameter values
    a : numpy.ndarray
        parameter values

    Returns
    -------
    Y : Result of G*-function
    """
    # Check the dimension of the input
    if (values.shape[1] != delta.shape[0]):
        raise ValueError("The dimension of inputs is not consistent")
    elif (values.shape[1] != alpha.shape[0]):
        raise ValueError("The dimension of inputs is not consistent")
    elif (values.shape[1] != a.shape[0]):
        raise ValueError("The dimension of inputs is not consistent")
    else:
        pass
    
    if type(values) != np.ndarray:
        raise TypeError("The argument `values` must be a numpy ndarray")

    ltz = values < 0
    gto = values > 1

    if ltz.any() == True:
        raise ValueError("Sobol G function called with values less than zero")
    elif gto.any() == True:
        raise ValueError("Sobol G function called with values greater than one")
        
    gi = ((1 + alpha) * np.power(np.abs(2 * (values + delta - np.modf(values + delta)[1]) - 1), alpha) + a) / (1 + a)

    return np.prod(gi, axis=1)

## Definde analytical parameter sensitivity indices for individual system model

In [3]:
def partial_first_order_variance(alpha, a):
    """Compute the partial first order variance of Vi

    Parameters
    ----------
    alpha : numpy.ndarray
        parameter values
    a : numpy.ndarray
        parameter values

    Returns
    -------
    Vi : The partial first order variance, which has the same size of alpah and a
    """
    return np.power(alpha, 2) / ((1 + 2 * alpha) * np.power((1 + a), 2))

def partial_total_order_variance(alpha, a):
    """Compute the partial total order variance of Vi

    Parameters
    ----------
    alpha : numpy.ndarray
        parameter values
    a : numpy.ndarray
        parameter values

    Returns
    -------
    VTi : The partial total order variance, which has the same size of alpah and a
    """
    pv = partial_first_order_variance(alpha, a)
    product_pv = np.product(1 + partial_first_order_variance(alpha, a), axis=0)
    return pv * np.divide(product_pv, 1 + pv.T)

def single_model_total_variance(alpha, a):
    """Compute the variance for a single system model

    Parameters
    ----------
    alpha : numpy.ndarray
        parameter values
    a : numpy.ndarray
        parameter values

    Returns
    -------
    V : The variance of the ouput, which is a scalar value 
    """
    return np.add(-1, np.product(1 + partial_first_order_variance(alpha, a), axis=0))

def first_order_parameter_sensitivity_index(alpha, a):
    """Compute the first order parameter sensitivity index

    Parameters
    ----------
    alpha : numpy.ndarray
        parameter values
    a : numpy.ndarray
        parameter values

    Returns
    -------
    Si: The first order parameter sensitivity index, which has the same size of alpah and a 
    """
    return np.divide(partial_first_order_variance(alpha, a), single_model_total_variance(alpha, a))

def total_parameter_sensitivity_index(alpha, a):
    """Compute the total parameter sensitivity index

    Parameters
    ----------
    alpha : numpy.ndarray
        parameter values
    a : numpy.ndarray
        parameter values

    Returns
    -------
    STi: The total parameter sensitivity index, which has the same size of alpah and a 
    """
    tv = single_model_total_variance(alpha, a)
    return np.divide(partial_total_order_variance(alpha, a), tv)

## Definde analytical process sensitivity indices

In [4]:
def multi_model_total_variance(Alpha, A):
    """Compute variance of the output considering mutiple system models

    Parameters
    ----------
    Alpha : numpy.ndarray
        parameter values
    A : numpy.ndarray
        parameter values

    Returns
    -------
    V : The variance of the ouput, which is a scalar value 
    """
    N = Alpha.shape[0]                    # Number of the system models
    model_weight = 1.0 / N                # Model weight for each system models
    return sum(single_model_total_variance(Alpha[i, :], A[i, :]) * model_weight for i in range(N))
    
def first_order_process_sensitivity_index(alpha, a):
    """Compute first-order process sensitivity index considering mutiple system models

    Parameters
    ----------
    alpha : numpy.ndarray
        parameter values
    a : numpy.ndarray
        parameter values

    Returns
    -------
    V : first-order process sensitivity index, which has the same size of alpah and a 
    """
    N = alpha.shape[0]                    # Number of processes
    return [np.mean(partial_first_order_variance(alpha[i, :], a[i, :])) / multi_model_total_variance(Alpha, A) for i in range(N)]

def total_order_process_sensitivity_index(Alpha, A):
    """Compute total process sensitivity index considering mutiple system models

    Parameters
    ----------
    Alpha : numpy.ndarray
        parameter values
    A : numpy.ndarray
        parameter values

    Returns
    -------
    V : first-order process sensitivity index, which has the same size of Alpha.shape[1]
    """
    N = Alpha.shape[0]                    # Number of the system models
    model_weight = 1 / N                  # Model weight for each system models
    return sum(partial_total_order_variance(Alpha[i, :], A[i, :]) * model_weight for i in range(N)) / multi_model_total_variance(Alpha, A)

## Define the parameters used in this case

In [5]:
# Three processes (or product elements) are considered here and each of which can be simulated by two alternative process models
delta = np.array([0, 0, 0])

alpha = np.array([[1,  2], [1, 2], [1, 2]])
a = np.array([[1.5, 1.2], [4.2, 1.8], [6.5, 2.3]])

# Generate the parameter combinations for the 8 system models
Alpha = np.array(list(itertools.product(* alpha)))
A = np.array(list(itertools.product(* a)))

## Compute the analytical parameter sensitity index for individual model 

In [6]:
theoretical_S1 = np.zeros([8, 3])
theoretical_ST = np.zeros([8, 3])
for i in range(8):
    theoretical_S1[i, :] = first_order_parameter_sensitivity_index(Alpha[i, :], A[i, :])
    theoretical_ST[i, :] = total_parameter_sensitivity_index(Alpha[i, :], A[i, :])
    
print('Model\tA1B1C1\t\t\tA1B1C2\t\t\tA1B2C1\t\t\tA1B2C2')
print('Para.\tx1\tx2\tx3\tx1\tx2\tx3\tx1\tx2\tx3\tx1\tx2\tx3')
print('S1\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f' %(theoretical_S1[0, 0] * 100, theoretical_S1[0, 1] * 100, theoretical_S1[0, 2] * 100, theoretical_S1[1, 0] * 100, theoretical_S1[1, 1] * 100, theoretical_S1[1, 2] * 100, theoretical_S1[2, 0] * 100, theoretical_S1[2, 1] * 100, theoretical_S1[2, 2] * 100, theoretical_S1[3, 0] * 100, theoretical_S1[3, 1] * 100, theoretical_S1[3, 2] * 100))
print('ST\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f' %(theoretical_ST[0, 0] * 100, theoretical_ST[0, 1] * 100, theoretical_ST[0, 2] * 100, theoretical_ST[1, 0] * 100, theoretical_ST[1, 1] * 100, theoretical_ST[1, 2] * 100, theoretical_ST[2, 0] * 100, theoretical_ST[2, 1] * 100, theoretical_ST[2, 2] * 100, theoretical_ST[3, 0] * 100, theoretical_ST[3, 1] * 100, theoretical_ST[3, 2] * 100))

print('Model\tA2B1C1\t\t\tA2B1C2\t\t\tA2B2C1\t\t\tA2B2C2')
print('Para.\tx1\tx2\tx3\tx1\tx2\tx3\tx1\tx2\tx3\tx1\tx2\tx3')
print('S1\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f' %(theoretical_S1[4, 0] * 100, theoretical_S1[4, 1] * 100, theoretical_S1[4, 2] * 100, theoretical_S1[5, 0] * 100, theoretical_S1[5, 1] * 100, theoretical_S1[5, 2] * 100, theoretical_S1[6, 0] * 100, theoretical_S1[6, 1] * 100, theoretical_S1[6, 2] * 100, theoretical_S1[7, 0] * 100, theoretical_S1[7, 1] * 100, theoretical_S1[7, 2] * 100))
print('ST\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f' %(theoretical_ST[4, 0] * 100, theoretical_ST[4, 1] * 100, theoretical_ST[4, 2] * 100, theoretical_ST[5, 0] * 100, theoretical_ST[5, 1] * 100, theoretical_ST[5, 2] * 100, theoretical_ST[6, 0] * 100, theoretical_ST[6, 1] * 100, theoretical_ST[6, 2] * 100, theoretical_ST[7, 0] * 100, theoretical_ST[7, 1] * 100, theoretical_ST[7, 2] * 100))


Model	A1B1C1			A1B1C2			A1B2C1			A1B2C2
Para.	x1	x2	x3	x1	x2	x3	x1	x2	x3	x1	x2	x3
S1	73.42	16.97	8.16	36.87	8.52	50.79	31.80	60.85	3.53	21.67	41.46	29.85
ST	74.77	17.98	8.70	40.07	9.64	54.15	35.26	64.47	4.10	25.64	46.88	34.65
Model	A2B1C1			A2B1C2			A2B2C1			A2B2C2
Para.	x1	x2	x3	x1	x2	x3	x1	x2	x3	x1	x2	x3
S1	88.56	6.60	3.17	62.07	4.63	27.58	56.64	34.97	2.03	43.67	26.96	19.41
ST	90.18	7.74	3.75	67.45	5.79	32.54	62.79	40.99	2.61	51.66	33.72	24.92


## Compute the analytical process sensitivity index considering process model uncertainty

In [7]:
print('Process\tA\tB\tC')
print('PS1\t%.2f\t%.2f\t%.2f' %(first_order_process_sensitivity_index(alpha, a)[0] * 100, first_order_process_sensitivity_index(alpha, a)[1] * 100, first_order_process_sensitivity_index(alpha, a)[2] * 100))
print('PST\t%.2f\t%.2f\t%.2f' %(total_order_process_sensitivity_index(Alpha, A)[0] * 100, total_order_process_sensitivity_index(Alpha, A)[1] * 100, total_order_process_sensitivity_index(Alpha, A)[2] * 100))

Process	A	B	C
PS1	49.85	26.08	18.10
PST	54.79	30.07	21.23


## Compute MC-based sensitivity index for each system model

The first-order and total parameter sensitivity index values for the individual system model are calculated using SALib - Sensitivity Analysis Library in Python. The package can be freely downloaded from https://salib.readthedocs.io/en/latest/.

In [8]:
# Define the model inputs
problem = {'num_vars': 3,
           'names': ['x1', 'x2', 'x3'],
           'bounds': [[0, 1], [0, 1], [0, 1]],
           'dists': ['unif', 'unif', 'unif']         
           }      

# Generate parameters           
param_values = saltelli.sample(problem, 40000, calc_second_order=False, seed=2**30) 

In [9]:
# Evaluate model
analytical_S1 = np.zeros([8, 3])
analytical_ST = np.zeros([8, 3])
for i in range(8):
    Y = evaluate(param_values, delta, Alpha[i, :], A[i, :])

    # Perform analysis
    sobol_Si = sobol.analyze(problem, Y, conf_level=0.95, print_to_console=False, calc_second_order=False)
    
    analytical_S1[i, :] = sobol_Si['S1']
    analytical_ST[i, :] = sobol_Si['ST']

print('Model\tA1B1C1\t\t\tA1B1C2\t\t\tA1B2C1\t\t\tA1B2C2')
print('Para.\tx1\tx2\tx3\tx1\tx2\tx3\tx1\tx2\tx3\tx1\tx2\tx3')
print('S1\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f' %(analytical_S1[0, 0] * 100, analytical_S1[0, 1] * 100, analytical_S1[0, 2] * 100, analytical_S1[1, 0] * 100, analytical_S1[1, 1] * 100, analytical_S1[1, 2] * 100, analytical_S1[2, 0] * 100, analytical_S1[2, 1] * 100, analytical_S1[2, 2] * 100, analytical_S1[3, 0] * 100, analytical_S1[3, 1] * 100, analytical_S1[3, 2] * 100))
print('ST\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f' %(analytical_ST[0, 0] * 100, analytical_ST[0, 1] * 100, analytical_ST[0, 2] * 100, analytical_ST[1, 0] * 100, analytical_ST[1, 1] * 100, analytical_ST[1, 2] * 100, analytical_ST[2, 0] * 100, analytical_ST[2, 1] * 100, analytical_ST[2, 2] * 100, analytical_ST[3, 0] * 100, analytical_ST[3, 1] * 100, analytical_ST[3, 2] * 100))

print('Model\tA2B1C1\t\t\tA2B1C2\t\t\tA2B2C1\t\t\tA2B2C2')
print('Para.\tx1\tx2\tx3\tx1\tx2\tx3\tx1\tx2\tx3\tx1\tx2\tx3')
print('S1\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f' %(analytical_S1[4, 0] * 100, analytical_S1[4, 1] * 100, analytical_S1[4, 2] * 100, analytical_S1[5, 0] * 100, analytical_S1[5, 1] * 100, analytical_S1[5, 2] * 100, analytical_S1[6, 0] * 100, analytical_S1[6, 1] * 100, analytical_S1[6, 2] * 100, analytical_S1[7, 0] * 100, analytical_S1[7, 1] * 100, analytical_S1[7, 2] * 100))
print('ST\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f' %(analytical_ST[4, 0] * 100, analytical_ST[4, 1] * 100, analytical_ST[4, 2] * 100, analytical_ST[5, 0] * 100, analytical_ST[5, 1] * 100, analytical_ST[5, 2] * 100, analytical_ST[6, 0] * 100, analytical_ST[6, 1] * 100, analytical_ST[6, 2] * 100, analytical_ST[7, 0] * 100, analytical_ST[7, 1] * 100, analytical_ST[7, 2] * 100))


Model	A1B1C1			A1B1C2			A1B2C1			A1B2C2
Para.	x1	x2	x3	x1	x2	x3	x1	x2	x3	x1	x2	x3
S1	73.43	16.97	8.16	36.87	8.52	50.78	31.81	60.85	3.53	21.66	41.47	29.86
ST	74.74	17.99	8.70	40.03	9.64	54.16	35.23	64.49	4.11	25.59	46.89	34.67
Model	A2B1C1			A2B1C2			A2B2C1			A2B2C2
Para.	x1	x2	x3	x1	x2	x3	x1	x2	x3	x1	x2	x3
S1	88.55	6.61	3.17	62.05	4.64	27.57	56.65	34.97	2.02	43.63	26.98	19.39
ST	90.15	7.74	3.75	67.40	5.79	32.54	62.75	40.99	2.61	51.57	33.72	24.95


### Compute the ouput considering both model uncertainty and parmateric uncertainty

In [10]:
# Model information
N = 800                # Number of samples generated for each  parameter
Ma = 2                 # Number of alterantive models for recharge process
Mb = 2                 # Number of alterantive models for geloogy process
Mc = 2                 # Number of alterantive models for snow melt process

# Generate parameters using SALib.sample.saltelli
problem = {'num_vars': 3,
           'names': ['x1', 'x2', 'x3'],
           'bounds': [[0, 1], [0, 1], [0, 1]],
           'dists': ['unif', 'unif', 'unif']         
           }    

param_values = saltelli.sample(problem, N, calc_second_order=False, seed=2**30)[::5, :]

# Calculate system output using numba to accelerate
@nb.njit(parallel=True, fastmath=True)
def cmpt_Dscs():
    sims = 0
    Y = np.zeros((Ma, N, Mb, N, Mc, N), dtype=np.float32)
    
    for i in range(Ma):
        for j in nb.prange(N):
            if i == 0:
                alpha1 = alpha[0, 0]
                a1 = a[0, 0]
                values = param_values[j, 0]
                g1 = ((1 + alpha1) * np.power(np.abs(2 * values - 1), alpha1) + a1) / (1 + a1)
            else:
                alpha1 = alpha[0, 1]
                a1 = a[0, 1]
                values = param_values[j, 0]
                g1 = ((1 + alpha1) * np.power(np.abs(2 * values - 1), alpha1) + a1) / (1 + a1)
                
            for k in range (Mb):
                for l in nb.prange(N):
                    if k == 0:
                        alpha2 = alpha[1, 0]
                        a2 = a[1, 0]
                        values = param_values[l, 1]
                        g2 = ((1 + alpha2) * np.power(np.abs(2 * values - 1), alpha2) + a2) / (1 + a2)
                    else:
                        alpha2 = alpha[1, 1]
                        a2 = a[1, 1]
                        values = param_values[l, 1]
                        g2 = ((1 + alpha2) * np.power(np.abs(2 * values - 1), alpha2) + a2) / (1 + a2)
                        
                    for m in range(Mc):
                        for n in nb.prange(N):
                            if m == 0:
                                alpha3 = alpha[2, 0]
                                a3 = a[2, 0]
                                values = param_values[n, 2]
                                g3 = ((1 + alpha3) * np.power(np.abs(2 * values - 1), alpha3) + a3) / (1 + a3)
                            else:
                                alpha3 = alpha[2, 1]
                                a3 = a[2, 1]
                                values = param_values[n, 2]
                                g3 = ((1 + alpha3) * np.power(np.abs(2 * values - 1), alpha3) + a3) / (1 + a3)
                            
                            sims = sims + 1
                            '''
                            # If parallel computing is used, do not print info 
                            if sims % (N * Mc * N) == 0 :
                                print('Evaluating dvds at i =', i, 'j =', j, 'k =' , k, 'l =', l, 'm =', m, 'n =', n)
                            '''
                            Y[i, j, k, l, m, n] = g1 * g2 * g3
                            
    return Y
                
# Save results to local disk
Y = cmpt_Dscs()
np.save('Y_Sobol_G_' + str(N) + '.npy', Y)

C:\ProgramData\Anaconda3\lib\site-packages\numba\np\ufunc\parallel.py:355: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 10005. The TBB threading layer is disabled.
  warnings.warn(problem)


### First-order process sensitivit index for g1 process

In [11]:
# @nb.jit(fastmath=True)
def SI_Process_A(Y):
    
    Ma = 2
    Mb = 2
    Mc = 2
    
    PMA = np.array([0.5, 0.5])
    PMB = np.array([0.5, 0.5])
    PMC = np.array([0.5, 0.5])
    
    Var_t_d = np.var(Y)
    
    E_tc_d = np.zeros((Ma, N, Mb, N, Mc))
    E_c_d = np.zeros((Ma, N, Mb, N))
    E_tb_d = np.zeros((Ma, N, Mb))
    E_b_d = np.zeros((Ma, N))
    E_ta_d = np.zeros(Ma)
    E_ta_d2 = np.zeros(Ma)
     
    for i in range(Ma):      
        for j in range(N):                
            for k in range(Mb):    
                for l in range(N):                 
                    for m in range(Mc):
                        E_tc_d[i, j, k, l, m] = np.mean(Y[i, j, k, l, m, :])
                    E_c_d[i, j, k, l] = PMC[0] * E_tc_d[i, j, k, l, 0] + PMC[1] * E_tc_d[i, j, k, l, 1]
                E_tb_d[i, j, k] = np.mean(E_c_d[i, j, k, :])
            E_b_d[i, j] = PMB[0] * E_tb_d[i, j, 0] + PMB[1] * E_tb_d[i, j, 1]
        E_ta_d[i] = np.mean(E_b_d[i, :])
        E_ta_d2[i] = np.mean(E_b_d[i, :]**2)
        
    E_a_d = PMA[0] * E_ta_d[0] + PMA[1] * E_ta_d[1]
    E_a_d2 = PMA[0] * E_ta_d2[0] + PMA[1] * E_ta_d2[1]
 
    Var_A = E_a_d2 - E_a_d**2
    SI_A = Var_A / (Var_t_d)
 
    return SI_A, E_b_d
   
SI_A, E_b_d = SI_Process_A(Y)

print('Firsd-order process sensitivity index for Process g1 is %.4f' %SI_A)

Firsd-order process sensitivity index for Process g1 is 0.4985


### First-order process sensitivit index for g2 process

In [12]:
# @nb.jit(fastmath=True)
def SI_Process_B(Y):
   
    Ma = 2
    Mb = 2
    Mc = 2
    
    PMA = np.array([0.5, 0.5])
    PMB = np.array([0.5, 0.5])
    PMC = np.array([0.5, 0.5])
    
    Var_t_d = np.var(Y)
    
    E_tc_d = np.zeros([Mb, N, Ma, N, Mc])
    E_c_d = np.zeros([Mb, N, Ma, N])
    E_ta_d = np.zeros([Mb, N, Ma])
    E_a_d = np.zeros([Mb, N])
    E_tb_d = np.zeros([Mb])
    E_tb_d2 = np.zeros([Mb])
     
    for i in range(Mb):      
        for j in range(N):                
            for k in range(Ma):    
                for l in range(N):                 
                    for m in range(Mc):
                        E_tc_d[i, j, k, l, m] = np.mean(Y[k, l, i, j, m, :])
                    E_c_d[i, j, k, l] = PMC[0] * E_tc_d[i, j, k, l, 0] + PMC[1] * E_tc_d[i, j, k, l, 1]
                E_ta_d[i, j, k] = np.mean(E_c_d[i, j, k, :])
            E_a_d[i, j] = PMA[0] * E_ta_d[i, j, 0] + PMA[1] * E_ta_d[i, j, 1]
        E_tb_d[i] = np.mean(E_a_d[i, :])
        E_tb_d2[i] = np.mean(E_a_d[i, :]**2)
        
    E_b_d = PMB[0] * E_tb_d[0] + PMB[1] * E_tb_d[1]
    E_b_d2 = PMB[0] * E_tb_d2[0] + PMB[1] * E_tb_d2[1]

    Var_B = E_b_d2 - E_b_d**2
    SI_B = Var_B / (Var_t_d)
 
    return SI_B, E_a_d
   
SI_B, E_a_d = SI_Process_B(Y)

print('Firsd-order process sensitivity index for Process g2 is %.4f' %SI_B)

Firsd-order process sensitivity index for Process g2 is 0.2609


### First-order process sensitivit index for g3 process

In [13]:
# @nb.jit(fastmath=True)
def SI_Process_C(Y):
    
    Ma = 2
    Mb = 2
    Mc = 2
    
    PMA = np.array([0.5, 0.5])
    PMB = np.array([0.5, 0.5])
    PMC = np.array([0.5, 0.5])
    
    Var_t_d = np.var(Y)
    
    E_tb_d = np.zeros([Mc, N, Ma, N, Mb])
    E_b_d = np.zeros([Mc, N, Ma, N])
    E_ta_d = np.zeros([Mc, N, Ma])
    E_a_d = np.zeros([Mc, N])
    E_tc_d = np.zeros([Mc])
    E_tc_d2 = np.zeros([Mc])
     
    for i in range(Mc):      
        for j in range(N):                
            for k in range(Ma):    
                for l in range(N):                 
                    for m in range(Mb):
                        E_tb_d[i, j, k, l, m] = np.mean(Y[k, l, m, :, i, j])
                    E_b_d[i, j, k, l] = PMB[0] * E_tb_d[i, j, k, l, 0] + PMB[1] * E_tb_d[i, j, k, l, 1]
                E_ta_d[i, j, k] = np.mean(E_b_d[i, j, k, :])
            E_a_d[i, j] = PMA[0] * E_ta_d[i, j, 0] + PMA[1] * E_ta_d[i, j, 1]
        E_tc_d[i] = np.mean(E_a_d[i, :])
        E_tc_d2[i] = np.mean(E_a_d[i, :]**2)
        
    E_c_d = PMC[0] * E_tc_d[0] + PMC[1] * E_tc_d[1]
    E_c_d2 = PMC[0] * E_tc_d2[0] + PMC[1] * E_tc_d2[1]

    Var_C = E_c_d2 - E_c_d**2
    SI_C = Var_C / (Var_t_d)
 
    return SI_C, E_a_d
   
SI_C, E_c_d = SI_Process_C(Y)

print('Firsd-order process sensitivity index for Process g3 is %.4f' %SI_C)

Firsd-order process sensitivity index for Process g3 is 0.1807


### Total process sensitivit index for g1 process

In [14]:
# @nb.jit(fastmath=True)
def ST_Process_A(Y):
    
    Ma = 2
    Mb = 2
    Mc = 2
    
    PMA = np.array([0.5, 0.5])
    PMB = np.array([0.5, 0.5])
    PMC = np.array([0.5, 0.5])
    
    Var_t_d = np.var(Y)
    
    E_ta_d = np.zeros([Mb, N, Mc, N, Ma])
    E_a_d = np.zeros([Mb, N, Mc, N])
    E_tc_d = np.zeros([Mb, N, Mc])
    E_tc_d2 = np.zeros([Mb, N, Mc])
    E_c_d = np.zeros([Mb, N])
    E_c_d2 = np.zeros([Mb, N])

    E_tb_d = np.zeros([Mb])
    E_tb_d2 = np.zeros([Mb])
    

    for i in range(Mb):
        for j in range(N):
            for k in range(Mc):
                for l in range(N):
                    for m in range(Ma):
                        E_ta_d[i, j, k, l, m] = np.mean(Y[m, :, i, j, k, l])
                    E_a_d[i, j, k, l] = PMA[0] * np.mean(E_ta_d[i, j, k, l, 0]) + PMA[1] * np.mean(E_ta_d[i, j, k, l, 1])
                E_tc_d[i, j, k] = np.mean(E_a_d[i, j, k, :])
                E_tc_d2[i, j, k] = np.mean(E_a_d[i, j, k, :]**2)
            E_c_d[i, j] = PMC[0] * E_tc_d[i, j, 0] + PMC[1] * E_tc_d[i, j, 1]
            E_c_d2[i, j] = PMC[0] * E_tc_d2[i, j, 0] + PMC[1] * E_tc_d2[i, j, 1]
        E_tb_d[i] = np.mean(E_c_d[i, :])
        E_tb_d2[i] = np.mean(E_tc_d2[i, :])
    E_b_d = PMB[0] * E_tb_d[0] + PMB[1] * E_tb_d[1]
    E_b_d2 = PMB[0] * E_tb_d2[0] + PMB[1] * E_tb_d2[1]
        
    Var_A = E_b_d2 - E_b_d**2
    ST_A = 1 - Var_A / (Var_t_d + 1e-20)
    
    return ST_A

ST_A = ST_Process_A(Y)

print('Total process sensitivity index for Process g1 is %.4f' %ST_A)    

Total process sensitivity index for Process g1 is 0.5480


### Total process sensitivity index for g2 process

In [15]:
# @nb.jit(fastmath=True)
def ST_Process_B(Y):
    
    Ma = 2
    Mb = 2
    Mc = 2
    
    PMA = np.array([0.5, 0.5])
    PMB = np.array([0.5, 0.5])
    PMC = np.array([0.5, 0.5])
    
    Var_t_d = np.var(Y)
    
    E_tb_d = np.zeros([Ma, N, Mc, N, Mb])
    E_b_d = np.zeros([Ma, N, Mc, N])
    E_tc_d = np.zeros([Ma, N, Mc])
    E_tc_d2 = np.zeros([Ma, N, Mc])
    E_c_d = np.zeros([Ma, N])
    E_c_d2 = np.zeros([Ma, N])

    E_ta_d = np.zeros([Ma])
    E_ta_d2 = np.zeros([Ma])
    

    for i in range(Ma):
        for j in range(N):
            for k in range(Mc):
                for l in range(N):
                    for m in range(Mb):
                        E_tb_d[i, j, k, l, m] = np.mean(Y[i, j, m, :, k, l])
                    E_b_d[i, j, k, l] = PMB[0] * np.mean(E_tb_d[i, j, k, l, 0]) + PMB[1] * np.mean(E_tb_d[i, j, k, l, 1])
                E_tc_d[i, j, k] = np.mean(E_b_d[i, j, k, :])
                E_tc_d2[i, j, k] = np.mean(E_b_d[i, j, k, :]**2)
            E_c_d[i, j] = PMC[0] * E_tc_d[i, j, 0] + PMC[1] * E_tc_d[i, j, 1]
            E_c_d2[i, j] = PMC[0] * E_tc_d2[i, j, 0] + PMC[1] * E_tc_d2[i, j, 1]
        E_ta_d[i] = np.mean(E_c_d[i, :])
        E_ta_d2[i] = np.mean(E_tc_d2[i, :])
    E_a_d = PMA[0] * E_ta_d[0] + PMA[1] * E_ta_d[1]
    E_a_d2 = PMA[0] * E_ta_d2[0] + PMA[1] * E_ta_d2[1]
        
    Var_B = E_a_d2 - E_a_d**2
    ST_B = 1 - Var_B / (Var_t_d + 1e-20)
    
    return ST_B

ST_B = ST_Process_B(Y)

print('Total process sensitivity index for Process g2 is %.4f' %ST_B) 

Total process sensitivity index for Process g2 is 0.3009


### Total process sensitivity index for snow-melt process

In [16]:
# @nb.jit(fastmath=True)
def ST_Process_C(Y):
    
    Ma = 2
    Mb = 2
    Mc = 2
    
    PMA = np.array([0.5, 0.5])
    PMB = np.array([0.5, 0.5])
    PMC = np.array([0.5, 0.5])
    
    Var_t_d = np.var(Y)
    
    E_tc_d = np.zeros([Ma, N, Mb, N, Mc])
    E_c_d = np.zeros([Ma, N, Mb, N])
    E_tb_d = np.zeros([Ma, N, Mb])
    E_tb_d2 = np.zeros([Ma, N, Mb])
    E_b_d = np.zeros([Ma, N])
    E_b_d2 = np.zeros([Ma, N])

    E_ta_d = np.zeros([Ma])
    E_ta_d2 = np.zeros([Ma])
    

    for i in range(Ma):
        for j in range(N):
            for k in range(Mb):
                for l in range(N):
                    for m in range(Mc):
                        E_tc_d[i, j, k, l, m] = np.mean(Y[i, j, k, l, m, :])
                    E_c_d[i, j, k, l] = PMC[0] * np.mean(E_tc_d[i, j, k, l, 0]) + PMC[1] * np.mean(E_tc_d[i, j, k, l, 1])
                E_tb_d[i, j, k] = np.mean(E_c_d[i, j, k, :])
                E_tb_d2[i, j, k] = np.mean(E_c_d[i, j, k, :]**2)
            E_b_d[i, j] = PMB[0] * E_tb_d[i, j, 0] + PMB[1] * E_tb_d[i, j, 1]
            E_b_d2[i, j] = PMB[0] * E_tb_d2[i, j, 0] + PMB[1] * E_tb_d2[i, j, 1]
        E_ta_d[i] = np.mean(E_b_d[i, :])
        E_ta_d2[i] = np.mean(E_tb_d2[i, :])
    E_a_d = PMA[0] * E_ta_d[0] + PMA[1] * E_ta_d[1]
    E_a_d2 = PMA[0] * E_ta_d2[0] + PMA[1] * E_ta_d2[1]
        
    Var_C = E_a_d2 - E_a_d**2
    ST_C = 1 - Var_C / (Var_t_d + 1e-20)
    
    return ST_C

ST_C = ST_Process_C(Y)

print('Total process sensitivity index for Process g3 is %.4f' %ST_C) 

Total process sensitivity index for Process g3 is 0.2120
